In [40]:
import numpy as np
import networkx as nx
import os
import pandas as pd
import torch
import time
import math
import dgl as dgl
from dgl.nn import pytorch
import numpy as np
import torch as th
import dgl.function as fn
import torch.nn.functional as F
from dgl.data import CoraGraphDataset
from GraphSAGE.losses import compute_loss_multiclass
from utils import *
from model import *
from loss import *
import matplotlib.cm as cm
import matplotlib.pyplot as plt



def train(g, features, n_classes, in_feats, n_edges, labels, mask, Q, args):
    # sethyperparameter
    dropout = 0.0
    gpu = 0
    early_stop_rate=0.000005
    loss_direction=1
    n_epochs = 1000
    n_hidden = features.shape[1]  # number of hidden nodes
    n_layers = 0  # number of hidden layers
    weight_decay_gamma = 0.65 #
    self_loop = True  #
    early_stop = False
    visualize_model=False
    last_score = 0

    grad_direction=args['grad_direction']
    lr = args['lr']
    cuda=args['cuda']
    nn_model = args['nn_model']


    #step_size=int(n_epochs/100)

    step_size = 2
    if self_loop:
        g = dgl.add_self_loop(g)
    # run single train of some model
    degs = g.in_degrees().float()
    norm = torch.pow(degs, -0.5)
    norm[torch.isinf(norm)] = 0

    if cuda:
        torch.cuda.set_device(gpu)
        features = features.cuda()
        labels = labels.cuda()
        g = g.to('cuda:0')
        mask = mask.cuda()

    if nn_model == 'GCN':
        model = eval(nn_model)(g,
                               in_feats,
                               n_hidden,
                               n_classes,
                               n_layers,
                               F.relu,
                               dropout)
        if cuda:
            model.cuda()
    else:
        model = eval(nn_model)(in_feat=in_feats,
                               out_feat=n_classes,
                               num_rels=3,
                               regularizer='basis',
                               num_bases=None,
                               bias=True,
                               activation=None,
                               self_loop=True,
                               low_mem=False,
                               dropout=0.0,
                               layer_norm=False
                               )
        if cuda:
            model.cuda()

    loss_fcn = ModularityScore(n_classes, cuda,loss_direction)


    if visualize_model:
        print_parameter(model)
        print_parameter(loss_fcn)

    #optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    #apply weight_decay scheduler
    #use self written D method
    #optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    #StepLR = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=weight_decay_gamma)
    optimizer=mySGD(model.parameters(),lr=lr,batch_size=features.shape[0],grad_direction=grad_direction)

    # train and evaluate (with modularity score and labels)
    dur = []

    print("initial inputs \n", features)
    print("#####################")



    for epoch in range(n_epochs):

        model.train()
        t0 = time.time()
        C_hat = model(features)
        #print('#############WX###################')
        #print(C_hat)
        # use train_mask to train
        loss = loss_fcn(C_hat[mask], Q)
        #if loss jump check parameters
        if  epoch==0:

            print("initial output WX : \n", C_hat)
            print('initial parameters and gradients')
            for i,p in enumerate(list(model.parameters())):
                print("param {}".format(i),p)
                print("param {} grad".format(i), p.grad)
            print('#####################')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #StepLR.step()

        dur.append(time.time() - t0)
        if epoch % step_size == 0:


        #if epoch % 1 == 0:
            # if visualize_model:
            #     print_parameter(model)
            #     print_parameter(loss_fcn)
            C_out,eval_loss = evaluate_M(C_hat,Q,cuda)
        if epoch == 0:
            #print('calculate the modularity from classic method')
            ground_truth_modularity = eval_loss
        print("Epoch {} | Time(s) {} |  True_Modularity {} | Ground_Truth_Modulairty {} | ETputs(KTEPS) {}".format(epoch, np.mean(dur),  (loss),
                                                                                                                                        ground_truth_modularity,
                                         n_edges / np.mean(dur) / 1000))
        if early_stop:
            if abs((loss - last_score) / last_score) < 0.000005 or torch.isnan(loss).sum()>0:
                loss=last_score
                C_hat=last_C_hat
                C_out, eval_loss = evaluate_M(C_hat, Q, cuda)

                print("Epoch {} | Time(s) {}  Train_Modularity {} | True_Modularity {}" |
                      "ETputs(KTEPS) {}".format(epoch, np.mean(dur), eval_loss, (loss),
                                                n_edges / np.mean(dur) / 1000))
                break


        last_score = loss
        last_C_hat=C_hat

    #C_out = C_construction(model, features, mask)

    C_init,modularity_init = evaluate_M(features, Q, cuda)
    print('initial modularity is', modularity_init)
    C_hat,modularity_score = evaluate_M(C_hat, Q, cuda)
    if torch.isnan(modularity_score):
        modularity_score=loss
    print(C_hat)
    return modularity_score.cpu().detach().numpy(), C_hat.cpu(), model.__str__(),features.cpu()

def main(nx_g, nn_model,grad_direction):
    # note g is a networkx class
    gpu = 0
    if gpu < 0:
        cuda = False
    else:
        cuda = True
    # prepare training data, set hyperparameters
    #config hyper_parameters
    lr_type = 'linspace'
    lr=np.linspace(5e-5,5e-1,num=10)
    args={
        'lr':lr,
        'grad_direction':grad_direction,
        'nn_model':nn_model,
        'cuda':cuda
    }
    g, features, n_classes, in_feats, n_edges, labels,Q,mask,modularity_classic = generate_model_input(nx_g,cuda)
    if len(lr)>1:
        print('learning_rate scanning mode for {} intervals from {} to {}'.format(len(lr),np.min(lr),np.max(lr)))
    for i, learning_rate  in enumerate(lr):
        train(g, features, n_classes, in_feats, n_edges, labels, mask, Q, args)
        if i==len(lr)-1:
            return train(g, features, n_classes, in_feats, n_edges, labels, mask, Q, args),modularity_classic





In [41]:
if __name__ == "__main__":
    test_number = 10
    work_dir = os.getcwd()
    nn_model = 'GCN'
    data_dir = os.path.join(work_dir, 'data/ComboSampleData/')
    # G = loadNetworkMat('karate_34.mat',data_dir)
    G = loadNetworkMat('celeganmetabolic_453.mat', data_dir)
    modularity_scores_gcn = {}
    nmi_gcn={}
    nmi={}
    C_init={}
    C_out = {}
    C_out_combo = {}
    graph_type = {}
    modularity_scores_combo = {}
    model_parameter = {}
    data_name = []
    modularity_scores_classic={}
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file[-3:] == 'mat':
                if file !='karate_34.mat':
                    continue
                data_name.append(file)
                G = loadNetworkMat(file, data_dir)
                if nx.classes.function.is_directed(G):
                    graph_type[file] = 'directed'
                else:
                    graph_type[file] = 'undirected'
                print(file, graph_type[file])

                # need to figure out it is weighted or not
                modularity_scores_combo[file], partition = getNewComboPartition(G)
                C_out_combo[file] = partition_to_binary_attachment(partition)
                [modularity_scores_gcn[file], C_out[file], model_parameter[file],C_init[file]],modularity_scores_classic[file] = main(G, nn_model,grad_direction=1)

                nmi_gcn[file]=NMI(C_out[file],C_init[file])
                nmi[file] = NMI(C_out_combo[file], C_init[file])



    ##save log
    save_result(data_name, graph_type, modularity_scores_gcn, modularity_scores_combo, modularity_scores_classic,nmi_gcn,nmi,model_parameter,
                data_dir)

    print('something')

karate_34.mat undirected
using self written gradient dec func
initial inputs 
 tensor([[5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [9.3473e-14, 5.5247e-04, 9.9890e-01, 5.5247e-04],
        [9.3473e-14, 5.5247e-04, 9.9890e-01, 5.5247e-04],
        [9.3473e-14, 5.5247e-04, 9.9890e-01, 5.5247e-04],
        [5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [9.3473e-14, 5.5247e-04, 9.9890e-01, 5.5247e-04],
        [5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [5.5247e-04, 9.9890e-01, 5.5247e-04, 9.3473e-14],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404

In [42]:
if __name__ == "__main__":
    test_number = 10
    work_dir = os.getcwd()
    nn_model = 'GCN'
    data_dir = os.path.join(work_dir, 'data/ComboSampleData/')
    # G = loadNetworkMat('karate_34.mat',data_dir)
    G = loadNetworkMat('celeganmetabolic_453.mat', data_dir)
    modularity_scores_gcn = {}
    nmi_gcn={}
    nmi={}
    C_init={}
    C_out = {}
    C_out_combo = {}
    graph_type = {}
    modularity_scores_combo = {}
    model_parameter = {}
    data_name = []
    modularity_scores_classic={}
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file[-3:] == 'mat':
                if file !='karate_34.mat':
                    continue
                data_name.append(file)
                G = loadNetworkMat(file, data_dir)
                if nx.classes.function.is_directed(G):
                    graph_type[file] = 'directed'
                else:
                    graph_type[file] = 'undirected'
                print(file, graph_type[file])

                # need to figure out it is weighted or not
                modularity_scores_combo[file], partition = getNewComboPartition(G)
                C_out_combo[file] = partition_to_binary_attachment(partition)
                [modularity_scores_gcn[file], C_out[file], model_parameter[file],C_init[file]],modularity_scores_classic[file] = main(G, nn_model,grad_direction=-1)

                nmi_gcn[file]=NMI(C_out[file],C_init[file])
                nmi[file] = NMI(C_out_combo[file], C_init[file])



    ##save log
    save_result(data_name, graph_type, modularity_scores_gcn, modularity_scores_combo, modularity_scores_classic,nmi_gcn,nmi,model_parameter,
                data_dir)

    print('something')


karate_34.mat undirected
using self written gradient dec func
initial inputs 
 tensor([[9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [9.3473e-14, 5.5247e-04, 9.9890e-01, 5.5247e-04],
        [9.3473e-14, 5.5247e-04, 9.9890e-01, 5.5247e-04],
        [9.3473e-14, 5.5247e-04, 9.9890e-01, 5.5247e-04],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [4.8404e-30, 9.3525e-14, 5.5278e-04, 9.9945e-01],
        [4.8404e-30, 9.3525e-14, 5.5278e-04, 9.9945e-01],
        [9.3473e-14, 5.5247e-04, 9.9890e-01, 5.5247e-04],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [9.9945e-01, 5.5278e-04, 9.3525e-14, 4.8404e-30],
        [4.8404e-30, 9.3525e-14, 5.5278e-04, 9.9945e-01],
        [4.8404e-30, 9.3525e-14, 5.5278e-04, 9.9945

C:\Users\benno\anaconda3\envs\gpu\lib\site-packages\dgl\base.py:45: DGLWarning: DGLGraph.__len__ is deprecated.Please directly call DGLGraph.number_of_nodes.
  return warnings.warn(message, category=category, stacklevel=1)


Epoch 44 | Time(s) 0.003200721740722656 |  True_Modularity 0.41902410984039307 | Ground_Truth_Modulairty 0.4197896122932434 | ETputs(KTEPS) 48.739007210535725
Epoch 45 | Time(s) 0.0031746314919513206 |  True_Modularity 0.4190170168876648 | Ground_Truth_Modulairty 0.4197896122932434 | ETputs(KTEPS) 49.13956167684614
Epoch 46 | Time(s) 0.0031496463937962307 |  True_Modularity 0.419014036655426 | Ground_Truth_Modulairty 0.4197896122932434 | ETputs(KTEPS) 49.529369489625495
Epoch 47 | Time(s) 0.0031257023413976035 |  True_Modularity 0.41900700330734253 | Ground_Truth_Modulairty 0.4197896122932434 | ETputs(KTEPS) 49.90878303858176
Epoch 48 | Time(s) 0.0031231909382100007 |  True_Modularity 0.4189978837966919 | Ground_Truth_Modulairty 0.4197896122932434 | ETputs(KTEPS) 49.94891541578579
Epoch 49 | Time(s) 0.003120684623718262 |  True_Modularity 0.41899561882019043 | Ground_Truth_Modulairty 0.4197896122932434 | ETputs(KTEPS) 49.989030873016475
Epoch 50 | Time(s) 0.0031183560689290366 |  True_